# Creating a model perceptron working on 2 variables Iris and saving into the file


In [2]:
%%file perceptron.py

#import of libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

#Model of perceptron
class Perceptron:
    
    def __init__(self, eta=0.1, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
    
    def predict(self, X):
        return np.where(self._net_input(X) >= 0, 1, -1)
    
    def _net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

#import of data
iris = load_iris()
iris.keys()

#data transformation
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], 
                  columns=iris['feature_names']+['target'])
df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df = df.drop(columns=['target'])
df['species'] = np.where(df['species'] == 'setosa',-1,1)
df = df[0:100]
df = df.sample(frac=1)
df_X = df.iloc[:,[0,2]]
df_y = df.iloc[:,4]

#division into train and test dataset
X_train = df_X[:int(df_X.shape[0]*0.7)]
X_test = df_X[int(df_X.shape[0]*0.7):]
y_train = df_y[:int(df_y.shape[0]*0.7)]
y_test = df_y[int(df_X.shape[0]*0.7):]

#import of the object
model = Perceptron()

#training of the perceptron
model.fit(X_train.values, y_train.to_numpy())

Overwriting perceptron.py


# Flask server allowing for a request with 2 params

In [3]:
import subprocess
import requests

In [4]:
%%file server.py

from flask import Flask
from flask import request
from perceptron import *


# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/hello', methods=['GET'])
def function():
    
    #parameters - user adds a name and decides if the w parameter is passed in response
    name = request.args.get("name", "")
    param_errors = bool(request.args.get("errors", ""))
    param_w = bool(request.args.get("w", ""))
    
    #parameters values
    if name:
        welcome = f"Hello {name}, here are your prediction results"
    else:
        welcome = f"Hello, here are your prediction results"
    prediction = model.predict(X_test).tolist()
    errors = model.errors_
    w = model.w_.tolist()
    
    #response
    if param_errors == True and param_w == True:
        resp1 = {'welcome': welcome, 'prediction': prediction, 'errors': errors, 'w' : w}
    elif param_errors == True and param_w != True:
        resp1 = {'welcome': welcome, 'prediction': prediction, 'errors': errors}
    elif param_errors != True and param_w == True:
        resp1 = {'welcome': welcome, 'prediction': prediction, 'w' : w}
    elif param_errors!= True and param_w != True:
        resp1 = {'welcome': welcome, 'prediction': prediction}
    
    #return
    return resp1

if __name__ == '__main__':
    app.run()

Overwriting zadanie.py


In [5]:
#open the port
p1 = subprocess.Popen(["python", "zadanie.py"])

 * Serving Flask app "zadanie" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


# Flask server response including model prediction and relevant data in a json format


In [6]:
#getting the response
response = requests.get("http://127.0.0.1:5000/hello?name=marcin&w=True&errors=True")
response.content

127.0.0.1 - - [27/Apr/2022 22:38:19] "GET /hello?name=marcin&w=True&errors=True HTTP/1.1" 200 -


b'{"errors":[7,0,0,0,0,0,0,0,0,0],"prediction":[1,1,-1,1,1,1,1,-1,-1,1,-1,-1,-1,1,-1,1,1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,-1,1],"w":[-0.2,-0.48,1.16],"welcome":"Hello marcin, here are your prediction results"}\n'

In [7]:
#kill the process
p1.kill()

# Saving the response to the database

In [8]:
import json
import pandas as pd
import numpy as np

json_response = json.loads(response.content)

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///perceptron.db')

In [10]:
from perceptron import *
df = X_test
df['prediction'] = json_response['prediction']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
#save to database
df.to_sql('perceptron2', con=engine, index=False)

In [13]:
#check if the results are in db
a = engine.execute('select prediction from perceptron2').fetchall()
b = engine.execute('select * from perceptron2').fetchall()
print (a)
print(b)

[(1,), (1,), (-1,), (1,), (1,), (1,), (1,), (-1,), (-1,), (1,), (-1,), (-1,), (-1,), (1,), (-1,), (1,), (1,), (-1,), (-1,), (1,), (-1,), (-1,), (1,), (1,), (1,), (-1,), (-1,), (-1,), (-1,), (1,)]
[(6.3, 4.9, 1), (5.1, 1.5, 1), (4.8, 1.6, -1), (5.5, 1.4, 1), (5.5, 4.0, 1), (5.4, 1.7, 1), (5.5, 3.8, 1), (4.4, 1.4, -1), (5.9, 4.2, -1), (5.1, 1.5, 1), (5.5, 4.0, -1), (5.8, 4.1, -1), (6.8, 4.8, -1), (5.1, 3.0, 1), (4.8, 1.6, -1), (5.6, 3.9, 1), (5.6, 3.6, 1), (4.9, 1.5, -1), (5.0, 1.4, -1), (5.1, 1.7, 1), (6.1, 4.0, -1), (6.0, 4.5, -1), (4.6, 1.4, 1), (5.3, 1.5, 1), (5.8, 1.2, 1), (6.3, 4.7, -1), (6.6, 4.6, -1), (5.7, 4.2, -1), (5.5, 3.7, -1), (5.0, 3.3, 1)]
